You are tasked with creating a neural network that HR can use to predict whether employees are likely to leave the company. Additionally, HR believes that some employees may be better suited to other departments, so you are also asked to predict the department that best fits each employee. These two columns should be predicted using a branched neural network.

## Part 1: Preprocessing

In [393]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [394]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [395]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [396]:
attrition_df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [397]:
# Create a list of at least 10 column names to use as X data
# list of categorical cols
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat.pop(0)
attrition_cat.pop(1)
attrition_cat

['BusinessTravel', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime']

In [398]:
attrition_df[attrition_cat].nunique()

,0
BusinessTravel,3
EducationField,6
JobRole,9
MaritalStatus,3
OverTime,2


In [399]:
# Create X_df using your selected columns
X_df = attrition_df.drop(columns=['Attrition', 'Department'])


# Show the data types for X_df
X_df.dtypes

,0
Age,int64
BusinessTravel,object
DistanceFromHome,int64
Education,int64
EducationField,object
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64
JobLevel,int64
JobRole,object


In [400]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [401]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1102 entries, 464 to 1061
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1102 non-null   int64 
 1   BusinessTravel            1102 non-null   object
 2   DistanceFromHome          1102 non-null   int64 
 3   Education                 1102 non-null   int64 
 4   EducationField            1102 non-null   object
 5   EnvironmentSatisfaction   1102 non-null   int64 
 6   HourlyRate                1102 non-null   int64 
 7   JobInvolvement            1102 non-null   int64 
 8   JobLevel                  1102 non-null   int64 
 9   JobRole                   1102 non-null   object
 10  JobSatisfaction           1102 non-null   int64 
 11  MaritalStatus             1102 non-null   object
 12  NumCompaniesWorked        1102 non-null   int64 
 13  OverTime                  1102 non-null   object
 14  PercentSalaryHike         1

In [402]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
X_train_encoded = pd.DataFrame(enc.fit_transform(X_train[attrition_cat]))
X_test_encoded = pd.DataFrame(enc.fit_transform(X_test[attrition_cat]))
# Add the encoded variable names to the dataframe
X_train_encoded.columns = enc.get_feature_names_out(attrition_cat)
X_test_encoded.columns = enc.get_feature_names_out(attrition_cat)

display(X_train_encoded.head())
display(X_test_encoded.head())

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [403]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_encoded).astype(np.float32)
X_test_scaled = scaler.transform(X_test_encoded).astype(np.float32)


In [404]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = encoder.transform(y_train[['Department']]).astype(np.int32)
y_test_department_encoded = encoder.transform(y_test[['Department']])



In [405]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = encoder.transform(y_train[['Attrition']]).astype(np.int32)
y_test_attrition_encoded = encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [406]:
# Find the number of columns in the X training data
input_nodes = len(X_train.columns)

# Create the input layer
from keras.layers import Input

input_layer = Input(shape=(X_train_scaled.shape[-1],))

# Create at least two shared layers
from keras.layers import Dense

shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

In [415]:
# Create a branch for Department
# with a hidden layer and an output layer
department_branch = Dense(32, activation='relu')(shared_layer_2)
# Create the hidden layer


# Create the output layer
department_output = Dense(3, activation='softmax', name='department_output')(department_branch)


In [419]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_branch = Dense(32, activation='relu')(shared_layer_2)
# Create the hidden layer


# Create the output layer
attrition_output = Dense(2, activation='sigmoid', name='attrition_output')(attrition_branch)


In [427]:
# Create the model
from keras.models import Model

model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'binary_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics=['accuracy', 'accuracy'])

# Summarize the model
model.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16            │ (None, 23)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_51 (Dense)          │ (None, 64)             │          1,536 │ input_layer_16[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_52 (Dense)          │ (None, 32)             │          2,080 │ dense_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_55 (Dense)          │ (None, 32)             │          1,056 │ dense_52[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_57 (Dense)          │ (None, 32)             │          1,056 │ dense_52[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dense_55[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dense_57[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,893 (23.02 KB)

 Trainable params: 5,893 (23.02 KB)

 Non-trainable params: 0 (0.00 B)

In [428]:
display(y_train_department_encoded.shape)
display(y_train_attrition_encoded.shape)
display(X_train_scaled.shape)

(1102, 3)

(1102, 2)

(1102, 23)

In [429]:
# Train the model
model.fit(X_train_scaled, {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded}, epochs=10, batch_size=32)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - attrition_output_accuracy: 0.8593 - department_output_accuracy: 0.9739 - loss: 0.3849
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8798 - department_output_accuracy: 0.9829 - loss: 0.3101
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8857 - department_output_accuracy: 0.9842 - loss: 0.3141
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8884 - department_output_accuracy: 0.9854 - loss: 0.3115
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8758 - department_output_accuracy: 0.9853 - loss: 0.3363
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8973 - department_output_accuracy: 0.9860 - loss: 0.3038
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8890 - department_output_accuracy: 0.9887 - loss: 0.3038
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━

In [434]:
# Evaluate the model with the testing data
department_output_accuracy, attrition_output_accuracy, loss = model.evaluate(X_test_scaled, {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded})



12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.7867 - department_output_accuracy: 0.9795 - loss: 0.5578


In [439]:
# Print the accuracy for both department and attrition
print(f'Department Accuracy: {round(department_output_accuracy,4)}, Attrition Accuracy: {round(attrition_output_accuracy,4)}')

Department Accuracy: 0.8043, Attrition Accuracy: 0.4942


# Summary

In the provided space below, briefly answer the following questions.

**1. Is accuracy the best metric to use on this data? Why or why not?**   
- Accuracy can be a risky metric for this dataset, as most employees likely remain with the company. Since the goal is to predict whether an employee will leave, accuracy alone might be misleading; if most employees stay, the model could appear highly accurate simply by predicting that employees will remain.
- A confusion matrix, precision, and recall would be good on this data as it details more information on true positive, false negatives, etc. 

**2. What activation functions did you choose for your output layers, and why?**   
- For the department output, I used the softmax activation function, which is suitable for multi-class outputs. Softmax returns a probability between 0 and 1 for each class, with probabilities summing to 1. For the attrition output, where the column has binary options, the sigmoid activation function was the best fit.

**3. Can you name a few ways that this model might be improved?**  
- To improve the model's evaluation, I suggest using additional metrics beyond accuracy to gauge performance. A confusion matrix, F1 score, and cross-validation scores would offer deeper insight into how well the model is performing. Additionally, we could try to streamline the data by encoding columns back into a single row to enhance model efficiency.